<h6><center>Big Data Algorithms Techniques & Platforms</center></h6>

<h1>
<hr style=" border:none; height:3px;">
<center>Assignment 1: Introduction to MapReduce</center>
<hr style=" border:none; height:3px;">
</h1>


# <strong> Important </strong>

For this assignment, as you did in the first lab, do not use the native fuctions <code>map()</code> and <code>reduce()</code> of Python.


<hr>



## Flights and Airports Data

For this assignment you are going to analyze a dataset that will include flight data.

The dataset comes from <a href="https://www.kaggle.com/flashgordon/usa-airport-dataset">Kaggle</a> and is presented here as an extract called <code>extract_flights.csv</code>

The dataset is in a <code>.csv</code> file. Each line of the file represents a different flight. The data collected contain:


<code>Origin_airport</code>: Three letter airport code of the origin airport </br>
<code>Destination_airport</code>: Three letter airport code of the destination airport</br>
<code>Origin_city</code>: Origin city name</br>
<code>Destination_city</code>: Destination city name</br>
<code>Passengers</code>: Number of passengers transported from origin to destination</br>
<code>Seats</code>: Number of seats available on flights from origin to destination</br>
<code>Flights</code>: Number of flights between origin and destination (multiple records for one month, many with flights > 1)</br>
<code>Distance</code>: Distance (to nearest mile) flown between origin and destination</br>
<code>Fly_date</code>: The date (yyyymm) of flight</br>
<code>Origin_population</code>: Origin city's population as reported by US Census</br>
<code>Destination_population</code>: Destination city's population as reported by US Census</br>


Two files are available in the folder:

<ul>
  <li> <code> small_file.csv</code> </li>
  <li> <code> medium_file.csv</code> </li>
</ul>

For more complex analysis/statistics you can also download the original file available on kaggle.



### Helper function ... just in case

Here is a function that you can modify, reuse, not-use for giving you an idea about how to read a <code>.csv</code> file and remove the header.

In [185]:
# Here a simple function that reads a .csv file, removes the header,
# and writes the output on a .txt file having the same name of the input file + the keyword "no_header"
# input: a csv file
def remove_header(filename):
    with open(filename + ".csv",'r') as f:
        with open(filename + "no_header.txt",'w') as f1:
            next(f) # skip header line
            for line in f:
                f1.write(line)
    return f1

remove_header('small_file')
print("file correctly saved")


file correctly saved


### <strong> Exercise 1 - Almost-empty flights</strong> 
#### <strong> 4 points </strong>
Describe and define a MapReduce procedure that gives as output the number of flights that departed with 10% of capacity maximum (i.e. empty, 1%, 6%, etc.) . 

The output can be of any form you like. 

You can use any data structure you want to support your implementation.




# Implementing everything from scratch without using any python library except data structures like list, dict,tuple,string.

In [186]:
arr_words = [line.split(",") for line in open('small_fileno_header.txt','r')]

In [187]:
def partition_fun(seq,num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

partition_list = partition_fun(arr_words,10) # Trying to create 10 partitions
print("Size odf each Partition is {}".format(len(partition_list[0]))) # Size of eaxch Paritiion
    

Size odf each Partition is 999


In [188]:
# Implementing everything from scratch without using any python library except data structures like list, dict,tuple,string.
def map_e_flights(arr):
    res = []
    for row in arr:
        if (row[2]+row[3]).replace(" ",'')!=(row[4]+row[5]).replace(" ",'') and int(row[7])!=0 :
            d = dict()
            d['percent_boarded'] = ((int(row[6])/int(row[7]))*100)
            d['num_flights']= int(row[8])
        res.append(d)
    return res

def shuffle_e_flights(ld):
    temp = {}
    Z = []
    for row in ld:
        k,v = row['percent_boarded'],row['num_flights']
        if k not in Z:
            temp[k]=[v]
            Z.append(k)
        else:
            temp[k].append(v)
    res = []
    for k,v in temp.items():
        d=dict()
        d['percent_boarded']=k
        d['num_flights']=v
        res.append(d)
            
    return res


def reduce_e_flights(ld):
    total_flights=[]
    for r in ld:
        k,v = r['percent_boarded'],r['num_flights']
        if k <10:
            total_flights.append(r)
    s=0
    for r in total_flights:
        s+=r['num_flights']
    return s

   
            


In [189]:
list_of_dict = map_e_flights(arr_words)
print("my map looks like following (only a small chunk is shown) : \n\n ", list_of_dict[:2])
shuffled_list = shuffle_e_flights(list_of_dict)# rearrange all keys togeteher and sorted them in increasing order
required_num_flights = reduce_e_flights(list_of_dict)
print("\n",required_num_flights)
# The answer could be varying a bit because of the division and calculating percentage capacity.

my map looks like following (only a small chunk is shown) : 

  [{'percent_boarded': 70.0, 'num_flights': 1}, {'percent_boarded': 10.353535353535353, 'num_flights': 22}]

 1269


Look at your implementation and think about your program. Is there any option to add a `combine` operation?  If yes describe your new approach and provide an improved version of your code.

Describe in detail your procedure before the code (in the comment lines).

### <strong> Exercise 2 - Top five destination airports </strong>
### <strong> 4 points </strong>

Provide now a function that lists the top five destination <strong>airports</strong>: the ones that have the highest number of incoming flights. Implement an algorithm that uses the MapReduce procedure.



In [191]:
#Provide here few comment lines that describe your map function

def map_top_flights(arr):
    res=[]
    for r in arr:
        d=dict()
        d['num_flights']=int(r[8])
        d['dest_airport']=r[1]
        res.append(d)
    return res
        
    

def shuffle_top_flights(ld):
    temp = {}
    Z = []
    for row in ld:
        k,v = row['dest_airport'],row['num_flights']
        if k not in Z:
            temp[k]=[v]
            Z.append(k)
        else:
            temp[k].append(v)
    res = []
    for k,v in temp.items():
        d=dict()
        d['dest_airport']=k
        d['num_flights']=v
        res.append(d)
            
    return res
    

def reduce_top_flights(ld):
    temp = []
    for pair in ld:
        total_flights=sum(pair['num_flights'])
        temp.append({'total_incoming_flights':total_flights,'airport_name':pair['dest_airport']})
    return sorted(temp,key = lambda x:x['total_incoming_flights'],reverse=True)[:5]
        

In [192]:
arr_words = [line.split(',') for line in open('small_fileno_header.txt','r')]
x1 = map_top_flights(arr_words)
x2 = shuffle_top_flights(x1)
x3 = reduce_top_flights(x2)
print(x3)
print('\n''The top five destination airports are')
[i['airport_name'] for i in x3]

[{'total_incoming_flights': 108593, 'airport_name': '"RDM"'}, {'total_incoming_flights': 25742, 'airport_name': '"EKO"'}, {'total_incoming_flights': 7101, 'airport_name': '"ERI"'}, {'total_incoming_flights': 3295, 'airport_name': '"WDG"'}, {'total_incoming_flights': 7, 'airport_name': '"END"'}]

The top five destination airports are


['"RDM"', '"EKO"', '"ERI"', '"WDG"', '"END"']

### <strong> Exercise 3 - Top 5 destination cities </strong>
#### <strong>  2 points </strong>

Try to reuse the code you run before and define a function that lists the top five destination <strong>cities</strong>: the ones that have the highest number of incoming flights. Implement an algorithm that uses the MapReduce procedure.


In [193]:
#Provide here few comment lines that describe your map function

def map_top_cities(arr):
    res=[]
    for r in arr:
        d=dict()
        d['num_flights']=int(r[8])
        d['dest_city']=r[4]+r[5].replace(' ','')
        res.append(d)
    return res
        
    
def shuffle_top_cities(ld):
    temp = {}
    Z = []
    for row in ld:
        k,v = row['dest_city'],row['num_flights']
        if k not in Z:
            temp[k]=[v]
            Z.append(k)
        else:
            temp[k].append(v)
    res = []
    for k,v in temp.items():
        d=dict()
        d['dest_city']=k
        d['num_flights']=v
        res.append(d)
            
    return res
    

def reduce_top_cities(ld):
    temp = []
    for pair in ld:
        total_flights=sum(pair['num_flights'])
        temp.append({'total_incoming_flights':total_flights,'city_name':pair['dest_city']})
    return sorted(temp,key = lambda x:x['total_incoming_flights'],reverse=True)[:5]
        

In [194]:
arr_words = [line.split(',') for line in open('small_fileno_header.txt','r')]
x1 = map_top_cities(arr_words)
x2 = shuffle_top_cities(x1)
x3 = reduce_top_cities(x2)
print(x3)
print('\n''The top five destination cities are')
[i['city_name'] for i in x3]

[{'total_incoming_flights': 108593, 'city_name': '"BendOR"'}, {'total_incoming_flights': 25742, 'city_name': '"ElkoNV"'}, {'total_incoming_flights': 7101, 'city_name': '"EriePA"'}, {'total_incoming_flights': 3302, 'city_name': '"EnidOK"'}, {'total_incoming_flights': 1, 'city_name': '"AmesIA"'}]

The top five destination cities are


['"BendOR"', '"ElkoNV"', '"EriePA"', '"EnidOK"', '"AmesIA"']

## Statistics on flights

<p align="justify">
<font size="3">
Now we want to run some more complex analysis on the flights. 
</font>
</p>





### <strong> Exercise 4 - Top five connections by month</strong>
#### <strong> 4 points </strong>

Try to reuse the code you run before and define now a function that lists the top five connections by each month: the top five pairs of cities that have the most number of connections (the function takes into account the flights from A to B and from B to A by month/year). Implement an algorithm that uses the MapReduce procedure.

In [196]:
#Provide here few comment lines that describe your map function

def map_top_connections(arr):
    res =[]
    for row in arr:
        city_pair = (row[2][1:]+row[3][1:-1],row[4][1:]+row[5][1:-1])
        year = row[10][:4]
        month = row[10][:4]
        num_flight = int(row[8])
        d = [num_flight,(city_pair,year,month)]
        res.append(d)
    return res

# def shuffle_top_connections(ld):
#     myset= set()
#     res=[]
#     for r in ld:
#         if r[1] not in myset:
#             myset.add(r[1])
#             res.append(r)
#         else:
            
            
        
           
     
        
        
            
        

# #Provide here few comment lines that describe your reduce function

# def reduce_top_connections( #to be completed

In [197]:
Z =map_top_connections(arr_words)[0:100]
Y=shuffle_top_connections(Z)
print(Y)

None


Look at your code. Can you use a <code>combine()</code> operation? 

If you cannot directly think about how you can represent the key-value pairs in a way that allows you to take advantage of a combine.

### <strong> Exercise 5 - Number of full flights</strong>
#### <strong> 2 points </strong>
<p align="justify">
<font size="3">
Describe and implement an algorithm that, following MapReduce procedure, shows how many full flights have departed. This exercise gives you an idea about how many times you can re-use code in MapReduce with minimum effort for repetitive analysis.
</font>
</p>



In [198]:
#Provide here few comment lines that describe your map function

#USABILITY FROM EXERCISE-1

def map_e_flights(arr):
    res = []
    for row in arr:
        if (row[4]+row[5]).replace(" ",'')!=(row[2]+row[3]).replace(" ",'') and int(row[7])!=0 :
            d = dict()
            d['percent_boarded'] = (round((int(row[6])/int(row[7]))*100,2))
            d['num_flights']= int(row[8])
            res.append(d)
    return res

def shuffle_e_flights(ld):
    temp = {}
    Z = []
    for row in ld:
        k,v = row['percent_boarded'],row['num_flights']
        if k not in Z:
            temp[k]=[v]
            Z.append(k)
        else:
            temp[k].append(v)
    res = []
    for k,v in temp.items():
        d=dict()
        d['percent_boarded']=k
        d['num_flights']=v
        res.append(d)
            
    return res

def reduce_full_flights(ld):
    res=[]
    for row in ld:
        if row['percent_boarded']==100.0:
            res.append(row)
    return res
    



In [200]:
arr_words = [line.split(',') for line in open('small_fileno_header.txt','r')]
x1 = map_e_flights(arr_words)
x2 = shuffle_e_flights(x1)
x3 = reduce_full_flights(x2)
s=0
for r in x3:
    s+=sum(r['num_flights'])
print(s)

554



### <strong> Exercise 6 -  Average/percentage </strong>
#### <strong> 4 points </strong>

<p align="justify">
<font size="3">
Describe and implement a MapReduce procedure that gives, for each city, the percentage/average, of full flights that have departed.

Notice that this exercise shares some similarities with one of the previous exercises. Think how and if you can modify (generalize) one of the functions already implemented before. 
</font>
</p>



In [201]:
# COPYING AND MODIFYING THE CODE FOR EXERCISE-3

def map_cities_capacity_flights(arr):
    res=[]
    for r in arr:
        d=dict()
        d['city']=r[2]+r[3].replace(" ",'')
        d['full_flights']=int(r[8]) if (int(r[6])==int(r[7])) and int(r[7]) else 0
        d['num_flights'] = int(r[8])
        res.append(d)
    return res
        
    
def shuffle_cities_capacity_flights(ld):
    temp= {}
    s1 = []
    for row in ld:
        ff,nf,ci= row['full_flights'],row['num_flights'],row['city']
        if ci not in s1:
            temp[ci]=[[ff],[nf]]
            s1.append(ci)
        else:
            temp[ci][0].append(ff)
            temp[ci][1].append(nf)
    res = []
    for k,v in temp.items():
        d=dict()
        d['city']=k
        d['total_full_flights']=sum(v[0])
        d['total_num_flights'] = sum(v[1])
        res.append(d)
            
    return res
    

def reduce_cities_full_flights(ld):
    res = []
    for row in ld:
        d = dict()
        d['city']=row['city']
        d['percent_full_flights'] = (row['total_full_flights']/row['total_num_flights'])*100
        res.append(d)
    return res
    
        
    
        

In [202]:
A = map_cities_capacity_flights(arr_words)
B = shuffle_cities_capacity_flights(A)
C = reduce_cities_full_flights(B)
C

[{'city': '"ManhattanKS"', 'percent_full_flights': 0.0},
 {'city': '"EugeneOR"', 'percent_full_flights': 0.333889816360601},
 {'city': '"MedfordOR"', 'percent_full_flights': 1.639344262295082},
 {'city': '"SeattleWA"', 'percent_full_flights': 0.015600624024960999},
 {'city': '"PortlandOR"', 'percent_full_flights': 0.001414727311310745},
 {'city': '"Klamath FallsOR"', 'percent_full_flights': 0.0},
 {'city': '"San FranciscoCA"', 'percent_full_flights': 0.0},
 {'city': '"Los AngelesCA"', 'percent_full_flights': 0.0},
 {'city': '"WenatcheeWA"', 'percent_full_flights': 0.0},
 {'city': '"YakimaWA"', 'percent_full_flights': 4.761904761904762},
 {'city': '"ElkoNV"', 'percent_full_flights': 2.272727272727273},
 {'city': '"SalemOR"', 'percent_full_flights': 4.3478260869565215},
 {'city': '"SpokaneWA"', 'percent_full_flights': 4.705882352941177},
 {'city': '"ReddingCA"', 'percent_full_flights': 0.0},
 {'city': '"LewistonID"', 'percent_full_flights': 6.363636363636363},
 {'city': '"AstoriaOR"', 'p

Look at your code. Can you use a <code>combine()</code> operation? 

If you cannot directly think about how you can represent the key-value pairs in a way that allows you to take advantage of a combine.

In [ ]:
#Provide here few comment lines that describe your map function

def  #to be completed

#Provide here few comment lines that describe your shuffle function

def  #to be completed

#Provide here few comment lines that describe your reduce function

def  #to be completed


### <strong> Bonus - Data Analysis </strong>
#### Data analysis

<p align="justify">
<font size="3">
Look at the number of almost empty flights and full flights and propose an analysis that explores if there is a correlation between the number of flights connecting two cities, the number of full flights and the percentage of almost empty flights. 

The granularity of the analysis is important: consider for example the week, the month and the year.
</font>
</p>


In [ ]:
# Define here your considerations and your code